In [1]:
# Setting up environment
import sys
import pandas as pd
sys.path.append("../../xapres_package/")
import ApRESDefs
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import importlib
import gcsfs

importlib.reload(ApRESDefs)
xa = ApRESDefs.xapres(loglevel='debug')

DEBUG    19:50:08 	 ApRESDefs.py @function _setup_logging line 495 - Stream logging level set to DEBUG
DEBUG    19:50:08 	 ApRESDefs.py @function _setup_logging line 496 - Add console handler to logger
DEBUG    19:50:08 	 ApRESDefs.py @function _setup_logging line 509 - File logging level set to DEBUG


In [2]:
# Loading site data
def reload(site):
    filename = f'gs://ldeo-glaciology/apres/greenland/2022/single_zarrs_noencode/{site}'
    ds = xr.open_dataset(filename,
        engine='zarr', 
        chunks={}) 
    return ds

ds_101 = reload("A101")

In [3]:
# Set up dask cluster to help with computation
from dask.distributed import Client

client = Client("tcp://127.0.0.1:38287")
client

<Client: 'tcp://127.0.0.1:38287' processes=4 threads=16, memory=58.87 GiB>

In [4]:
timeseries = ds_101.isel(attenuator_setting_pair = 0,time=range(954,1142)).profile_stacked.compute()


In [5]:
b1 = timeseries.isel(time=range(0,len(timeseries.time)-1))
b2 = timeseries.isel(time=range(1,len(timeseries.time)))

In [25]:
f = b1
g =b2 
win_cor=19 
step=19 
range_ext=None
win_wrap=0
thresh=0.9

In [31]:

t1 = f.time.data
t2 = g.time.data
dt = (t2-t1)/ np.timedelta64(1, 's')
dr = np.mean(np.diff(f.profile_range))
# configurations
maxStrain = 0.005
maxDepth = 1400
minDepth = 10
chunkWidth = 15

stepSizeM = chunkWidth
binStart = np.arange(minDepth,maxDepth,stepSizeM)

for ii in range(len(binStart)):
    depthRange = [binStart[ii], binStart[ii] + chunkWidth];
    binDepth = np.mean(depthRange)
    fi = find((f.rangeCoarse>=min(depthRange) & f.rangeCoarse<max(depthRange)));
    fc = f.profile_range.where(f.profile_range >= min(depthRange)

In [32]:
binDepth

1397.5